In [18]:
database = "T:/Occurrence_Records/footprintWKT.sqlite"
outFile = "T:/Occurrence_Records/footprintWKT.shp"
footprints = True

In [19]:
from datetime import datetime
import sqlite3
import pandas as pd
import geopandas as gpd
timestamp = datetime.now()

# Get the record coordinates as a data frame
records = (pd.read_sql("""SELECT * FROM occurrence_records;""",
                       con=sqlite3.connect(database))
                      .astype({'decimalLongitude': 'float',
                               'decimalLatitude': 'float',
                               'radius_meters': 'float'}))

# Make a point geometry from coordinates
gdf = gpd.GeoDataFrame(records,
                       geometry=gpd.points_from_xy(records['decimalLongitude'],
                                                   records['decimalLatitude']))

# Set the coordinate reference system
gdf.crs={'init' :'epsg:4326'}

# If user just wants shapefile of given coordinates (point method)
if footprints == False:
    # Simply save points
    gdf.to_file(outFile)

In [43]:
# If the user wants to identify record footprints
if footprints == True:
    # Reproject record coordinates to facilitate buffering
    feetprints = gdf.to_crs(epsg=5070)
    
    # Use point-radius method
    feetprints['footprint'] = feetprints.apply(lambda x: x.geometry.buffer(x.radius_meters), axis=1)
    feetprints.set_geometry(col='footprint', inplace=True, drop=True)

    # Overwrite when footprint WKT is provided (shape method; careful with projection) FORTHCOMING   
    if list(gdf['footprintWKT'].unique()) != ['nan']:
        print("ALERT! footprintWKT was provided but not used.")
    
    # Save results as shapefile
    feetprints.to_file(outFile)

print("Exported shapefile: " + str(datetime.now() - timestamp))

ALERT! footprintWKT was provided but not used.


UnicodeEncodeError: 'latin-1' codec can't encode character '\u2013' in position 4: ordinal not in range(256)